In [6]:
import glob
import pandas as pd
from verstack.stratified_continuous_split import scsplit
import shutil
import concurrent
import subprocess
import os

XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/amandeep/Github/table-linker/tl_env/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


In [4]:
home_dir = "/Users/amandeep/Github/table-linker/data/SemTab2020/Round4"
canonical_dir = f"{home_dir}/canonical"
output = subprocess.check_output('find '+HOME_DIR+' -name "*.csv" -exec wc -l {} \;', shell=True)

NameError: name 'subprocess' is not defined

In [4]:
data = output.decode("utf-8").split("\n")

In [5]:
data = data[:-1]
cleaned_data = list()
for l in data:
    temp = l.strip().split(" ")
    if os.path.exists("dataSplitTask/round3/gt/"+temp[1].split("/")[-1]):
        cleaned_data.append([temp[1], int(temp[0])])

In [6]:
df = pd.DataFrame(cleaned_data, columns=["File","Lines"])

In [7]:
train, test = scsplit(df, stratify = df["Lines"], test_size = 0.1, train_size = 0.9)

In [8]:
train =  pd.concat([df,test.reset_index(drop=True)]).drop_duplicates(keep=False).reset_index(drop=True)
percentage_100 = 100/train.shape[0]
train_12, df_100 = scsplit(train, stratify = train["Lines"], test_size = percentage_100, train_size = 1-percentage_100)

In [9]:
df_100 = df_100.reset_index(drop=True)
percentage_49 = 49/df_100.shape[0]
train_12 =  pd.concat([train,df_100.reset_index(drop=True)]).drop_duplicates(keep=False).reset_index(drop=True)
temp, dev = scsplit(df_100, stratify=df_100["Lines"], test_size = percentage_49, train_size=1-percentage_49)
train_12 = pd.concat([train_12, temp.reset_index(drop=True)]).reset_index(drop=True)

In [10]:
percentage_200 = 200/train_12.shape[0]
train_2, train1 = scsplit(train_12, stratify = train_12["Lines"], test_size = percentage_200, train_size = 1-percentage_200)

In [11]:
print(dev.shape)
print(train1.shape)
print(train_2.shape)
print(test.shape)
print(train1.shape[0]+train_2.shape[0]+dev.shape[0]+test.shape[0])

(49, 2)
(200, 2)
(56103, 2)
(6262, 2)
62614


In [12]:
test_path = HOME_DIR+"test"
!mkdir -p $test_path
for f in test["File"].tolist():
    shutil.move(f, test_path)

In [13]:
dev_path = HOME_DIR+"dev"
!mkdir -p $dev_path
for f in dev["File"].tolist():
    shutil.move(f, dev_path)

In [14]:
train1_path = HOME_DIR+"train1"
!mkdir -p $train1_path
for f in train1["File"].tolist():
    shutil.move(f, train1_path)

In [15]:
# train2 = train2.reset_index(drop=True)
train2_path = HOME_DIR+"train2"
!mkdir -p $train2_path
for f in train_2["File"].tolist():
    shutil.move(f, train2_path)